In [1]:
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 203.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 260.2 MB/s eta 0:00:00
DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from datetime import datetime
import os
import wandb
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#체크포인트 경로 설정 및 폴도 만들기
CHECKPOINT_FILE_PATH = os.path.join(os.getcwd(), 'checkpoints_classification2')

if not os.path.isdir(CHECKPOINT_FILE_PATH):
    os.makedirs(CHECKPOINT_FILE_PATH)

In [4]:
from utils.utils import strfdelta
from utils.early_stopping import EarlyStopping

class ClassificationTrainer:
  def __init__(
    self, project_name, model, optimizer, train_data_loader, validation_data_loader, transforms,
    run_time_str, wandb, device, checkpoint_file_path
  ):
    self.project_name = project_name
    self.model = model
    self.optimizer = optimizer
    self.train_data_loader = train_data_loader
    self.validation_data_loader = validation_data_loader
    self.transforms = transforms
    self.run_time_str = run_time_str
    self.wandb = wandb
    self.device = device
    self.checkpoint_file_path = checkpoint_file_path

    # Use a built-in loss function
    self.loss_fn = nn.CrossEntropyLoss()

  def do_train(self):
    self.model.train()  # Will be explained at 'Diverse Techniques' section

    loss_train = 0.0
    num_corrects_train = 0
    num_trained_samples = 0
    num_trains = 0

    for train_batch in self.train_data_loader:
      input_train, target_train = train_batch
      input_train = input_train.to(device=self.device)
      target_train = target_train.to(device=self.device)

      if self.transforms:
        input_train = self.transforms(input_train)

      output_train = self.model(input_train)
      loss = self.loss_fn(output_train, target_train)
      loss_train += loss.item()

      predicted_train = torch.argmax(output_train, dim=-1)

      # >>> predicted_train: tensor([5, 8, 9, 0, 9, 8, 9, 8, ..., 0, 1, 3, 7, 1, 4, 3])
      # >>> target_train:    tensor([5, 8, 9, 2, 9, 8, 7, 8, ..., 4, 1, 9, 6, 1, 4, 3])
      num_corrects_train += torch.sum(torch.eq(predicted_train, target_train)).item()

      num_trained_samples += len(input_train)
      num_trains += 1

      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()

    train_loss = loss_train / num_trains
    train_accuracy = 100.0 * num_corrects_train / num_trained_samples

    return train_loss, train_accuracy

  def do_validation(self):
    self.model.eval()   # Explained at 'Diverse Techniques' section

    loss_validation = 0.0
    num_corrects_validation = 0
    num_validated_samples = 0
    num_validations = 0

    with torch.no_grad():
      for validation_batch in self.validation_data_loader:
        input_validation, target_validation = validation_batch
        input_validation = input_validation.to(device=self.device)
        target_validation = target_validation.to(device=self.device)

        if self.transforms:
          input_validation = self.transforms(input_validation)

        output_validation = self.model(input_validation)
        loss_validation += self.loss_fn(output_validation, target_validation).item()

        predicted_validation = torch.argmax(output_validation, dim=1)
        num_corrects_validation += torch.sum(torch.eq(predicted_validation, target_validation)).item()

        num_validated_samples += len(input_validation)
        num_validations += 1

    validation_loss = loss_validation / num_validations
    validation_accuracy = 100.0 * num_corrects_validation / num_validated_samples

    return validation_loss, validation_accuracy

  def train_loop(self):
    early_stopping = EarlyStopping(
      patience=self.wandb.config.early_stop_patience,
      delta=self.wandb.config.early_stop_delta,
      project_name=self.project_name,
      checkpoint_file_path=self.checkpoint_file_path,
      run_time_str=self.run_time_str
    )
    n_epochs = self.wandb.config.epochs
    training_start_time = datetime.now()

    for epoch in range(1, n_epochs + 1):
      train_loss, train_accuracy = self.do_train()

      if epoch == 1 or epoch % self.wandb.config.validation_intervals == 0:
        validation_loss, validation_accuracy = self.do_validation()

        elapsed_time = datetime.now() - training_start_time
        epoch_per_second = 0 if elapsed_time.seconds == 0 else epoch / elapsed_time.seconds

        message, early_stop = early_stopping.check_and_save(validation_loss, self.model)

        print(
          f"[Epoch {epoch:>3}] "
          f"T_loss: {train_loss:7.5f}, "
          f"T_accuracy: {train_accuracy:6.4f} | "
          f"V_loss: {validation_loss:7.5f}, "
          f"V_accuracy: {validation_accuracy:6.4f} | "
          f"{message} | "
          f"T_time: {strfdelta(elapsed_time, '%H:%M:%S')}, "
          f"T_speed: {epoch_per_second:4.3f}"
        )

        self.wandb.log({
          "Epoch": epoch,
          "Training loss": train_loss,
          "Training accuracy (%)": train_accuracy,
          "Validation loss": validation_loss,
          "Validation accuracy (%)": validation_accuracy,
          "Training speed (epochs/sec.)": epoch_per_second,
        })

        if early_stop:
          break

    elapsed_time = datetime.now() - training_start_time
    print(f"Final training time: {strfdelta(elapsed_time, '%H:%M:%S')}")


In [5]:
def get_cryptocurrency_data(
    sequence_size=10, validation_size=100, test_size=10, target_column='Close', y_normalizer=1.0e7, is_regression=True
):
  btc_krw_path = os.path.join(os.getcwd(), "BTC_KRW.csv")
  df = pd.read_csv(btc_krw_path)
  row_size = len(df)
  # ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
  date_list = df['Date']

  df = df.drop(columns=['Date'])

  data_size = row_size - sequence_size
  train_size = data_size - (validation_size + test_size)
  #################################################################################################

  row_cursor = 0

  X_train_list = []
  y_train_regression_list = []
  y_train_classification_list = []
  y_train_date = []
  for idx in range(0, train_size):
    sequence_data = df.iloc[idx: idx + sequence_size].values  # sequence_data.shape: (sequence_size, 5)
    X_train_list.append(torch.from_numpy(sequence_data))
    y_train_regression_list.append(df.iloc[idx + sequence_size][target_column])
    y_train_classification_list.append(
      1 if df.iloc[idx + sequence_size][target_column] >= df.iloc[idx + sequence_size - 1][target_column] else 0
    )
    y_train_date.append(date_list[idx + sequence_size])
    row_cursor += 1

  X_train = torch.stack(X_train_list, dim=0).to(torch.float)
  y_train_regression = torch.tensor(y_train_regression_list, dtype=torch.float32) / y_normalizer
  y_train_classification = torch.tensor(y_train_classification_list, dtype=torch.int64)

  m = X_train.mean(dim=0, keepdim=True)
  s = X_train.std(dim=0, keepdim=True)
  X_train = (X_train - m) / s

  #################################################################################################

  X_validation_list = []
  y_validation_regression_list = []
  y_validation_classification_list = []
  y_validation_date = []
  for idx in range(row_cursor, row_cursor + validation_size):
    sequence_data = df.iloc[idx: idx + sequence_size].values  # sequence_data.shape: (sequence_size, 5)
    X_validation_list.append(torch.from_numpy(sequence_data))
    y_validation_regression_list.append(df.iloc[idx + sequence_size][target_column])
    y_validation_classification_list.append(
      1 if df.iloc[idx + sequence_size][target_column] >= df.iloc[idx + sequence_size - 1][target_column] else 0
    )
    y_validation_date.append(date_list[idx + sequence_size])
    row_cursor += 1

  X_validation = torch.stack(X_validation_list, dim=0).to(torch.float)
  y_validation_regression = torch.tensor(y_validation_regression_list, dtype=torch.float32) / y_normalizer
  y_validation_classification = torch.tensor(y_validation_classification_list, dtype=torch.int64)

  X_validation = (X_validation - m) / s
  #################################################################################################

  X_test_list = []
  y_test_regression_list = []
  y_test_classification_list = []
  y_test_date = []
  for idx in range(row_cursor, row_cursor + test_size):
    sequence_data = df.iloc[idx: idx + sequence_size].values  # sequence_data.shape: (sequence_size, 5)
    X_test_list.append(torch.from_numpy(sequence_data))
    y_test_regression_list.append(df.iloc[idx + sequence_size][target_column])
    y_test_classification_list.append(
      1 if df.iloc[idx + sequence_size][target_column] > df.iloc[idx + sequence_size - 1][target_column] else 0
    )
    y_test_date.append(date_list[idx + sequence_size])
    row_cursor += 1

  X_test = torch.stack(X_test_list, dim=0).to(torch.float)
  y_test_regression = torch.tensor(y_test_regression_list, dtype=torch.float32) / y_normalizer
  y_test_classification = torch.tensor(y_test_classification_list, dtype=torch.int64)

  X_test = (X_test - m) / s

  if is_regression:
    return (
      X_train, X_validation, X_test,
      y_train_regression, y_validation_regression, y_test_regression,
      y_train_date, y_validation_date, y_test_date
    )
  else:
    return (
      X_train, X_validation, X_test,
      y_train_classification, y_validation_classification, y_test_classification,
      y_train_date, y_validation_date, y_test_date
    )

In [6]:
class CryptoCurrencyDataset(Dataset):
  def __init__(self, X, y, is_regression=True):
    self.X = X
    self.y = y

    assert len(self.X) == len(self.y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    X = self.X[idx]
    y = self.y[idx]
    return X, y

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

In [7]:
def get_btc_krw_data(sequence_size=21, validation_size=150, test_size=30, is_regression=True):
  X_train, X_validation, X_test, y_train, y_validation, y_test, y_train_date, y_validation_date, y_test_date = get_cryptocurrency_data(
    sequence_size= sequence_size, validation_size=validation_size, test_size=test_size, target_column='Close', y_normalizer=1.0e7, is_regression=is_regression
  ) #데이터 분할
  
  #데이터를 커스텀 dataset에 저장
  train_crypto_currency_dataset = CryptoCurrencyDataset(X=X_train, y=y_train)
  validation_crypto_currency_dataset = CryptoCurrencyDataset(X=X_validation, y=y_validation)
  test_crypto_currency_dataset = CryptoCurrencyDataset(X=X_test, y=y_test)

  #dataset을 dataloader에 저장해 배치 단위로 사용할 수 있게 함
  train_data_loader = DataLoader(
    dataset = train_crypto_currency_dataset,batch_size=wandb.config.batch_size,shuffle=True
  )
  validation_data_loader = DataLoader(
    dataset = validation_crypto_currency_dataset,batch_size=wandb.config.batch_size,shuffle=True
  )
  test_data_loader = DataLoader(
    dataset=test_crypto_currency_dataset,batch_size=len(test_crypto_currency_dataset),shuffle=True
  )
  #데이터 로더 반환
  return train_data_loader, validation_data_loader, test_data_loader

In [8]:
def get_model():
  class MyModel(nn.Module):
    def __init__(self, n_input, n_output, p):
      super().__init__()
      
      self.lstm=nn.LSTM(input_size=n_input, hidden_size=128, num_layers=9, batch_first = True)
      self.fcn = nn.Linear(in_features=128, out_features = 64)
      self.fcn2 = nn.Linear(in_features=64, out_features = 32)
      self.fcn3 = nn.Linear(in_features=32, out_features = n_output)
      self.dropout = nn.Dropout(p=p)
    def forward(self, x):
      x, hidden = self.lstm(x)
      x = x[:,-1,:]
      x = self.fcn(x)
      x = self.dropout(x)
      x = self.fcn2(x)
      x = self.dropout(x)
      x = self.fcn3(x)
      return x
  
  my_model = MyModel(n_input= 5, n_output= 2, p=0.05)
  
  return my_model

In [9]:
def main_train(epochs, batch_size, validation_intervals, learning_rate ,early_stop_patience, early_stop_delta, weight_decay):
  
  run_time_str = datetime.now().astimezone().strftime("%Y_%m_%d_%H_%M_%S")
  
  config = {
    'epochs': epochs,
    'batch_size': batch_size,
    'validation_intervals': validation_intervals,
    'learning_rate': learning_rate,
    'early_stop_patience': early_stop_patience,
    'early_stop_delta': early_stop_delta,
    'weight_decay': weight_decay
  }
  
  project_name = 'lstm_classification_btc_krw'
  wandb.init(
    mode = 'online',
    project  = project_name,
    notes = 'btc_krw experiment with lstm',
    tags = ['lstm', ' classification', 'btc_krw'],
    name = run_time_str,
    config=config
  )
  
  train_data_loader, validation_data_loader, _ = get_btc_krw_data(is_regression = False)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model = get_model()
  model.to(device)
  
  optimizer = optim.Adam(model.parameters(), lr = wandb.config.learning_rate, weight_decay=wandb.config.weight_decay)
  
  classification_trainer = ClassificationTrainer(
    project_name, model, optimizer, train_data_loader, validation_data_loader, None, run_time_str, wandb, device, CHECKPOINT_FILE_PATH
  )
  classification_trainer.train_loop()
  
  wandb.finish()

In [10]:
def test(test_model):
  _, _, test_data_loader = get_btc_krw_data(is_regression=False)

  test_model.eval()

  num_corrects_test = 0
  num_tested_samples = 0

  print("[TEST DATA]")
  with torch.no_grad():
    for test_batch in test_data_loader:
      input_test, target_test = test_batch

      output_test = test_model(input_test)

      predicted_test = torch.argmax(output_test, dim=1)
      num_corrects_test += torch.sum(torch.eq(predicted_test, target_test))

      num_tested_samples += len(input_test)

    test_accuracy = 100.0 * num_corrects_test / num_tested_samples

    print(f"TEST RESULTS: {test_accuracy:6.3f}%")

    for idx, (output, target) in enumerate(zip(output_test, target_test)):
      print("{0:2}: {1:6,.2f} <--> {2:6,.2f}".format(
        idx, torch.argmax(output).item(), target.item()
      ))

In [11]:
def main_test(epochs, batch_size, validation_intervals, learning_rate ,early_stop_patience, early_stop_delta):
  run_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')
  
  config = {
    'epochs': epochs,
    'batch_size': batch_size,
    'validation_intervals': validation_intervals,
    'learning_rate': learning_rate,
    'early_stop_patience': early_stop_patience,
    'early_stop_delta': early_stop_delta,
  }
  
  project_name = 'lstm_classification_btc_krw'
  wandb.init(
    mode="disabled",
    project=project_name,
    notes="btc_krw experiment with lstm",
    tags=["lstm", "regression", "btc_krw"],
    name=run_time_str,
    config=config
  )
  
  test_model = get_model()
  
  latest_file_path = os.path.join(
    CHECKPOINT_FILE_PATH, f"{project_name}_checkpoint_latest.pt"
  )
  
  print("MODEL FILE: {0}".format(latest_file_path))
  test_model.load_state_dict(torch.load(latest_file_path, map_location=torch.device('cpu')))
  
  test(test_model)

In [12]:
if __name__ == '__main__':
    epochs = 1000
    batch_size = 64
    validation_intervals = 10
    learning_rate = 1e-3
    early_stop_patience = 100
    early_stop_delta = 0.00001
    weight_decay  = 0.001
  
    main_train(epochs, batch_size, validation_intervals, learning_rate,early_stop_patience, early_stop_delta, weight_decay)
    main_test(epochs, batch_size, validation_intervals, learning_rate, early_stop_patience, early_stop_delta)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/work/.netrc
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


[Epoch   1] T_loss: 0.69198, T_accuracy: 52.9629 | V_loss: 0.68976, V_accuracy: 52.6667 | Early stopping is stated! | T_time: 00:00:01, T_speed: 1.000
[Epoch  10] T_loss: 0.69165, T_accuracy: 52.9629 | V_loss: 0.69126, V_accuracy: 52.6667 | Early stopping counter: 1 out of 100 | T_time: 00:00:09, T_speed: 1.111
[Epoch  20] T_loss: 0.69068, T_accuracy: 52.9629 | V_loss: 0.69362, V_accuracy: 52.6667 | Early stopping counter: 2 out of 100 | T_time: 00:00:18, T_speed: 1.111
[Epoch  30] T_loss: 0.69183, T_accuracy: 52.9629 | V_loss: 0.69243, V_accuracy: 52.6667 | Early stopping counter: 3 out of 100 | T_time: 00:00:27, T_speed: 1.111
[Epoch  40] T_loss: 0.69112, T_accuracy: 52.9629 | V_loss: 0.69243, V_accuracy: 52.6667 | Early stopping counter: 4 out of 100 | T_time: 00:00:37, T_speed: 1.081
[Epoch  50] T_loss: 0.69163, T_accuracy: 52.9629 | V_loss: 0.69702, V_accuracy: 52.6667 | Early stopping counter: 5 out of 100 | T_time: 00:00:46, T_speed: 1.087
[Epoch  60] T_loss: 0.69139, T_accuracy

[Epoch 510] T_loss: 0.69187, T_accuracy: 52.9629 | V_loss: 0.69236, V_accuracy: 52.6667 | Early stopping counter: 3 out of 100 | T_time: 00:04:52, T_speed: 1.747
[Epoch 520] T_loss: 0.69108, T_accuracy: 52.9629 | V_loss: 0.69255, V_accuracy: 52.6667 | Early stopping counter: 4 out of 100 | T_time: 00:04:57, T_speed: 1.751
[Epoch 530] T_loss: 0.69184, T_accuracy: 52.9629 | V_loss: 0.69562, V_accuracy: 52.6667 | Early stopping counter: 5 out of 100 | T_time: 00:05:01, T_speed: 1.761
[Epoch 540] T_loss: 0.69133, T_accuracy: 52.9629 | V_loss: 0.68765, V_accuracy: 52.6667 | Early stopping counter: 6 out of 100 | T_time: 00:05:06, T_speed: 1.765
[Epoch 550] T_loss: 0.69135, T_accuracy: 52.9629 | V_loss: 0.68887, V_accuracy: 52.6667 | Early stopping counter: 7 out of 100 | T_time: 00:05:11, T_speed: 1.768
[Epoch 560] T_loss: 0.69187, T_accuracy: 52.9629 | V_loss: 0.69242, V_accuracy: 52.6667 | Early stopping counter: 8 out of 100 | T_time: 00:05:16, T_speed: 1.772
[Epoch 570] T_loss: 0.69107,

Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
Training accuracy (%),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Training loss,▅▆▅▃▃▄▃▆▄█▁▆▅▅▅▃▅▃▅▆▃▁▅▃▂▃▃▁▅▃▅▅▁▃▅▃▃▃▄▃
Training speed (epochs/sec.),▁▁▁▁▁▂▃▃▃▅▅▅▅▆▆▆▆▇▇▇▇▇███████▇▆▆▆▆▆▅▅▅▅▅
Validation accuracy (%),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation loss,▄▅▄▄█▆▄▂▄▃▅▅▅▄▃▄▇▄▃▅▅▄▃▃▄▁▄▃▃▅▃▄▃▄▄▄▄▄▄▄
Epoch,1000
Training accuracy (%),52.96286
Training loss,0.69162
Training speed (epochs/sec.),1.4881
Validation accuracy (%),52.66667


MODEL FILE: /home/work/DL/4/checkpoints_classification2/lstm_classification_btc_krw_checkpoint_latest.pt
[TEST DATA]
TEST RESULTS: 53.333%
 0:   1.00 <-->   0.00
 1:   1.00 <-->   1.00
 2:   1.00 <-->   1.00
 3:   1.00 <-->   0.00
 4:   1.00 <-->   0.00
 5:   1.00 <-->   0.00
 6:   1.00 <-->   1.00
 7:   1.00 <-->   1.00
 8:   1.00 <-->   0.00
 9:   1.00 <-->   0.00
10:   1.00 <-->   1.00
11:   1.00 <-->   0.00
12:   1.00 <-->   0.00
13:   1.00 <-->   0.00
14:   1.00 <-->   1.00
15:   1.00 <-->   0.00
16:   1.00 <-->   1.00
17:   1.00 <-->   1.00
18:   1.00 <-->   1.00
19:   1.00 <-->   0.00
20:   1.00 <-->   1.00
21:   1.00 <-->   0.00
22:   1.00 <-->   1.00
23:   1.00 <-->   0.00
24:   1.00 <-->   1.00
25:   1.00 <-->   1.00
26:   1.00 <-->   1.00
27:   1.00 <-->   0.00
28:   1.00 <-->   1.00
29:   1.00 <-->   1.00


1. num_layers: 2 -> 4   : 53.333
2. batch_size: 32 -> 64 : 53.333
-------------------------------
#### 2번에서 나온 결과 -> 전부 1로 예측
-------------------------------
3. num_layers: 4 -> 2    : 46.667
-------------------------------
#### 3번에서 나온 결과 -> 전부 0으로 예측
-------------------------------
4. num_layers: 2 -> 3    : 46.667
-------------------------------
#### 4번에서 나온 결과 -> 전부 0으로 예측
-------------------------------
5. num_layers: 3->2, fcn늘리기(128>64>32>n_output) :53.333
-------------------------------
#### 5번에서 나온 결과 -> 0과 1을 번갈아가면서 예측 / 잘못함 6번 이후로 알았는데 init에서 추가하고 forward에는 추가를 안함 -> 추가안한것
-------------------------------
6. weight_decay: 0.001 -> 0.0001 : 53.333
-------------------------------
#### 6번에서 나온 결과 -> 전부 1로 예측
-------------------------------
7. weight_decay: 0.0001 -> 0.001, fcn과 dropout(0.2)추가: 53.333
-------------------------------
#### 7번에서 나온 결과 -> 전부 1로 예측
-------------------------------
8. early_stop_patience: 10 -> 100 : 53.333
-------------------------------
#### 8번에서 나온 결과 -> 전부 1로 예측
-------------------------------
9. dropout 0.2 -> 0.05 : 53.333
-------------------------------
#### 9번에서 나온 결과 -> 전부 1로 예측
-------------------------------
10. num_layers 2->9 : 53.333
-------------------------------
#### 10번에서 나온 결과 -> 전부 1로 예측
#### 뭔가 문제가 있는 것 같은데...
    데이터에 문제가 있나?
-------------------------------


## 숙제 후기

 문제가 있는지 1번 과제때부터 계속해서 하나의 영역으로만 분류하는 문제가 있어 문제를 해결하지 못했다.
 조금 일찍 시작해서 교수님이나 조교님에게 물어볼 시간을 확보했어야 했는데 너무 늦게 시작했다는 생각이 들었다.